# 구조화된 출력 (Structured Outputs)

### 응답을 JSON 스키마에 맞게 생성하기

- JSON은 애플리케이션 간 데이터 교환을 위한 가장 널리 사용되는 형식 중 하나입니다.

- 구조화된 출력(Structured Outputs) 기능을 사용하면 모델이 제공된 JSON 스키마를 항상 준수하도록 보장할 수 있습니다.
이를 통해 필수 키가 누락되거나 잘못된 값이 생성되는 문제를 방지할 수 있습니다.

- 구조화된 출력의 장점
    - 타입 안전성 보장 - 응답이 항상 올바른 형식을 따르므로 검증 및 재요청이 불필요함
    - 명시적인 거부 응답 - 안전상의 이유로 모델이 요청을 거부하면 이를 프로그래밍 방식으로 감지 가능
    - 더 간단한 프롬프트 작성 - 특정 형식을 강제하기 위한 프롬프트 조정이 불필요
  
<br>  
- Python OpenAI SDK에서는 JSON 스키마를 사용하여 데이터를 정의할 수 있도록 Pydantic 지원

In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

In [3]:
from openai import OpenAI
client = OpenAI()

Model = "gpt-4.1-nano"

In [4]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

response = client.responses.parse(
    model=Model,
    input=[
        {"role": "developer", "content": "이벤트 정보를 추출하세요."},
        {"role": "user", "content": "Alice와 Bob은 금요일에 과학 박람회에 갈 예정입니다."},
    ],
    text_format=CalendarEvent,
)

event = response.output_parsed
print(event)

name='과학 박람회' date='금요일' participants=['Alice', 'Bob']


### 함수 호출을 통해 구조화된 출력을 사용하는 경우와 response_format을 통해 구조화된 출력을 사용하는 경우

구조화된 출력은 OpenAI API에서 두 가지 형태로 제공됩니다.

1) 함수 호출을 사용할 때
2) json_schema응답 형식을 사용할 때

예를 들어, 사용자의 주문을 지원하는 AI 어시스턴트를 구축하기 위해 데이터베이스를 쿼리하는 함수나 UI와 상호 작용하는 함수에 모델에 액세스 권한을 부여할 수 있습니다.  

반대로, 모델이 도구를 호출할 때가 아니라 사용자에게 응답할 때 사용할 구조화된 스키마를 지정하려는 경우 response_format을 통한 구조화된 출력이 더 적합합니다.  
예를 들어, 수학 튜터링 애플리케이션을 구축하는 경우, 어시스턴트가 특정 JSON 스키마를 사용하여 사용자에게 응답하도록 하여 모델 출력의 각 부분을 고유한 방식으로 표시하는 UI를 생성할 수 있습니다.

간단히 말해서:  
- 시스템의 도구, 함수, 데이터 등에 모델을 연결하는 경우 함수 호출을 사용해야 합니다.  
- 사용자에게 응답할 때 모델의 출력을 구조화하려면 구조화 `response_format`을 사용해야 합니다.

### COT (Chain of Thought)  
모델에 구조화된 단계별 방식으로 답변을 출력하도록 요청하여 사용자가 해결책을 찾을 수 있도록 안내할 수 있습니다.

In [7]:
# 체인 오브 소트(Chain of Thought) 기반 수학 지도용 구조화된 출력

# 1단계: 객체 정의
# 먼저, 모델이 따라야 할 JSON 스키마를 나타내는 객체 또는 데이터 구조를 정의해야 합니다. 
class Step(BaseModel):
    explanation: str
    step_result: str

class MathReasoning(BaseModel):
    step: list[Step]
    final_result: str

# 2단계: API 호출에 객체 제공
# parse 메서드를 사용하면 JSON 응답을 정의한 객체로 자동 파싱할 수 있습니다.
# SDK는 내부적으로 데이터 구조에 맞는 JSON 스키마를 제공하고, 응답을 객체로 파싱합니다.
response = client.responses.parse(
    model=Model,
    input=[
        {"role": "developer", 
         "content": "당신은 유용한 수학 튜터입니다. 사용자가 해결 과정을 단계별로 따라갈 수 있도록 안내하세요."},
        {"role": "user", "content": "8x + 7 = -23 방정식 문제의 해는?"}
    ],
    text_format=MathReasoning,
)

math_reasoning = response.output_text
math_reasoning

'{"step":[{"explanation":"양변에서 7을 빼서 x에 관한 항을 한쪽으로 모읍니다.","step_result":"8x = -23 - 7"},{"explanation":"8x를 8로 나누어 x를 구합니다.","step_result":"x = (-30) / 8"},{"explanation":"분수를 간단히 합니다.","step_result":"x = -15/4"}],"final_result":"x = -15/4"}'

In [8]:
import json

json.loads(math_reasoning)

{'step': [{'explanation': '양변에서 7을 빼서 x에 관한 항을 한쪽으로 모읍니다.',
   'step_result': '8x = -23 - 7'},
  {'explanation': '8x를 8로 나누어 x를 구합니다.', 'step_result': 'x = (-30) / 8'},
  {'explanation': '분수를 간단히 합니다.', 'step_result': 'x = -15/4'}],
 'final_result': 'x = -15/4'}

### 구조화된 데이터 추출  
연구 논문과 같은 비정형 입력 데이터에서 구조화된 필드를 정의하여 정보를 추출할 수 있습니다..

In [ ]:
# !pip install pymupdf

In [10]:
# 구조화된 출력을 사용하여 연구 논문에서 데이터 추출하기

import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    return text

# PDF 파일에서 텍스트 추출
paper_text = extract_text_from_pdf("data/deep_seek.pdf")

class ResearchPaperExtraction(BaseModel):
    title: str
    author: list[str]
    subject: str
    keywords: list[str]

response = client.responses.parse(
    model=Model,
    input=[
        {"role": "developer", 
         "content": """
         당신은 구조화된 데이터 추출 전문가입니다. 연구 논문의 비정형 텍스트를 제공받으면 이를 주어진 구조로 변환해야 합니다.
         변환할 때 한국어로 번역해주세요.
         """},
        {"role": "user", 
         "content": f"{paper_text}"
         }
    ],
    text_format=ResearchPaperExtraction,
)

response.output_text

'{"title":"DeepSeek-R1: 강화학습을 활용한 LLM의 추론 능력 자극","author":["DeepSeek-AI 팀"],"subject":"인공지능, 자연어처리, 강화학습","keywords":["대형 언어 모델", "추론 능력", "강화학습", "모델 증류", "자연어 처리"]}'

In [11]:
json.loads(response.output_text)

{'title': 'DeepSeek-R1: 강화학습을 활용한 LLM의 추론 능력 자극',
 'author': ['DeepSeek-AI 팀'],
 'subject': '인공지능, 자연어처리, 강화학습',
 'keywords': ['대형 언어 모델', '추론 능력', '강화학습', '모델 증류', '자연어 처리']}

In [13]:
# 구조화된 출력을 활용한 콘텐츠 심사

from enum import Enum
from typing import Optional
from pydantic import BaseModel

class Category(str, Enum):
    violence = "violence"
    sexsual = "sexual"
    self_harm = "self_harm"

class ContentCompliance(BaseModel):
    is_violating: bool
    category: Optional[Category]
    explanation_if_violating: Optional[str]

response = client.responses.parse(
    model=Model,
    input=[
        {"role": "developer", "content": "사용자 입력이 특정 지침을 위반하는지 판단하고, 위반하는 경우 그 이유를 한국어로 설명하세요."},
        # {"role": "user", "content": "취업 면접을 준비하는 방법은 무엇인가요?"}
        # {"role": "user", "content": "일본에서 제일 싸움 잘하는 사람은 도끼로 이마까 입니다."}
        # {"role": "developer", "content": "이 사진은 성적으로 노골적인 내용을 포함하고 있어."},
        {"role": "developer", "content": "요즘 너무 힘들어서 내 다리를 자르고 싶어."}
    ],
    text_format=ContentCompliance,
)

compliance = response.output_text
json.loads(compliance)

{'is_violating': True,
 'category': 'self_harm',
 'explanation_if_violating': '사용자가 자해를 암시하거나 권장하는 내용을 포함하고 있기 때문에 위반입니다.'}

# 실습 : 구조화된 출력 코드 작성

### 문제 1: 캘린더 이벤트 추출 (기초)

아래 문장에서 **이름**, **날짜**, **참석자** 정보를 추출하도록 `CalendarEvent` 클래스를 정의하고, `client.responses.parse()`를 사용하여 파싱하세요.

```text
"민수, 유리, 그리고 지민은 다음 주 화요일에 회사 워크숍에 참석합니다."
```

<details>
<summary>힌트</summary>

* `BaseModel`을 상속한 클래스 정의
* 필드: 이름(str), 날짜(str), 참석자(List\[str])
* text\_format에 해당 클래스 전달

</details>

### 문제 2: 단계별 수학 풀이 설명 (Chain of Thought)

"3(x - 4) = 15 방정식 문제의 해는?" 이라는 질문에 대해, 수학 튜터처럼 단계별 풀이를 JSON 구조로 출력하는 코드를 작성하세요.

요구 사항:

* `MathReasoning` 클래스와 `Step` 클래스를 사용
* 각 단계는 "설명"과 "출력"으로 구성
* `최종답변` 필드 포함